# Week 5 - Natural Language Processing (NLP)

In this week's exercise, we will apply neural networks to a new type of data - text. You will also pick your own corpus of text to play with.  **But first, some definitions** (there's a lot of new jargon in 'NLP'):
* *Corpus*: The set of all text documents you want to work on
* *Document*: An individual unit of text in your corpus

Below are some **examples of corpora** to get you thinking:

* Corpus of 100,000 IMDB reviews, where each document is an individual review
* Corpus of 20 English novels, where each document is an individual novel
* Corpus of one Malay novel, where each document is a chapter

You can **browse for a corpus in the below links** (if you don't already have one in mind):

* https://github.com/niderhoff/nlp-datasets
* https://www.gutenberg.org/catalog/

If this is your first time working with text, it's probably easier to deal with a corpus of many short documents - for example the IMDB review dataset, which is linked below in Chapter 2.  Play around with several corpora over the course of week, and work with something that interests you. Remember text doesn't have to be English (try other languages), or even a natural language (try code or musical notation)!

**Key learning resources** for the week:
* https://web.stanford.edu/~jurafsky/slp3/ - legendary textbook introducing key theory and concepts of working with text, up to deep learning methods
* http://web.stanford.edu/class/cs224n/ - great course that introduces theory and concepts of text processing in the context of deep learning (can read class notes / assignments and skip videos if you are short on time) 
* https://course.fast.ai/index.html - fast.ai's introduction to deep learning (you'll have to pick out the bits about text and RNNs) is an efficient and effective way of tackling the topic
* https://www.datacamp.com/courses/natural-language-processing-fundamentals-in-python - very hands on datacamp course that will let you practice using existing tools for NLP tasks

Some **additional tools below** that can help in NLP (if you haven't found them already):
* scikit-learn has a handy set of features for NLP
* https://spacy.io/ - commercially oriented python package for NLP
* https://www.nltk.org/ - slightly more academic oriented python package for NLP 

## Imports

In [1]:
import numpy as np  # a conventional alias
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from src.preprocess import getFileNames
from scipy.cluster import  hierarchy
from scipy.cluster.hierarchy import linkage,dendrogram,cophenet
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation

# Chapter 1: How do we turn text into data we can use?

### Convert your corpus into bags of words

We can't apply any of the techniques we have learned over the past few weeks directly on raw text.  Therefore, our first task is to convert our corpus into numbers.  The simplest way to do this is to use a **bag of words**. You can see some examples of this here: https://liferay.de.dariah.eu/tatom/index.html.

Once you understand the concept, convert your corpus and documents into bags of words below:

- Using the IMDB movies reviews
- Unsup dataset is used

In [2]:
# mypath='/home/hanifa/workspace/AIAP/AIAP_Week5/aclImdb/train/tt'
mypath='/home/jupyter/AIAP_Week5/aclImdb/train/unsup/'
filenames=getFileNames(mypath)

In [3]:
def checkVectorizer(dtm):
    a=dtm.sum(axis=0)
    a.sort(axis=0)
    print(a[-100:])

- Was having memory errors loading all the unsup data, thus just took a sample of 7K.

In [10]:
from sklearn.feature_extraction import text 
my_additional_stop_words=['br','film','movie']
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

vectorizer = CountVectorizer(input='filename',stop_words=stop_words,max_df=1000)
# vectorizer = CountVectorizer(input='filename',stop_words='english')'
dtm = vectorizer.fit_transform(filenames)  # a sparse matrix 
vocab = vectorizer.get_feature_names()

# type(dtm)                                         
dtm = dtm.toarray()  # convert to a regular array 
vocab = np.array(vocab)

In [11]:
checkVectorizer(dtm)
dtm.shape

[ 445  446  446  447  449  449  451  453  454  457  457  467  468  473
  475  480  483  483  493  494  501  502  504  513  514  514  532  535
  536  537  538  540  543  547  547  554  556  560  564  573  573  583
  589  591  602  607  617  620  622  632  638  639  642  654  658  668
  681  685  691  692  705  715  721  723  724  747  749  761  767  771
  789  797  811  820  834  844  877  878  880  893  910  913  920  926
 1015 1038 1078 1086 1105 1147 1167 1169 1193 1249 1251 1262 1273 1345
 1361 1370]


(4445, 33431)

### Show us your bags

Show and explain what one of your documents looks like as a bag of words below.  What are the advantages and disadvantages of encoding text as bags of words?

- The column represents the words and the rows the document itself. 
- By summing up the column, we can total frequency across all documents

In [12]:
def getFrequentWord(dtm):
    dtm_sum=dtm.sum(axis=0)
    idx=dtm_sum.argmax(axis=0)
    print("Most frequent word: %s" %(vocab[idx]))

In [13]:
getFrequentWord(dtm)

Most frequent word: love


In [ ]:
# words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
# words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

Cons:
 - Most frequent words are considered the most important
 - In this case, the br word 

### Tell us a story with your bags
Now that your text is in a more digestible format, you can apply previously learned techniques to better understand the corpus. **Create a brief story around your corpus, for example by using clustering techniques.** Some examples of what you can do below:
* Use *Hierarchical Clustering* to understand similarity of documents in your corpus. What distance measure works best? Are the results what you expect?

* Use Hierarchical Clustering to understand similarity of documents in your corpus. What distance measure works best? Are the results what you expect?
- Comparing Euclidean and Cosine, Cosine seems better as it <br/>
[Hierarchial Clustering Code](https://datascience.stackexchange.com/questions/22828/clustering-with-cosine-similarity/22834)

In [ ]:
def doHierarchialClustering(dtm,metric,threshold):
    Z = hierarchy.linkage(dtm,"average", metric=metric)
    fig=plt.figure(figsize = (7,4))
    ax=fig.add_subplot(111)
    ax.set(title="method_str",xlabel='Clusters',ylabel='Height')
    dendrogram(Z,
    #            labels=df1.index.values,
           ax=ax,
           truncate_mode='lastp',
           orientation='top',
           show_leaf_counts=True
    )

    plt.show()
    C = hierarchy.fcluster(Z,threshold, criterion="distance")
    print("Number of clusters %d" %(len(np.unique(hierarchy.fcluster(Z, threshold, criterion="distance")))))
    c, coph_dists = cophenet(Z, pdist(dtm))
#     print ("My name is %s and weight is %d kg!" % ('Zara', 21))
    print("Cophenet score is %f"%(c))
    

In [ ]:
doHierarchialClustering(dtm,'euclidean',threshold=20)

In [ ]:
doHierarchialClustering(dtm,'cosine',threshold=.89)

* Learn about *Latent Dirichlet Allocation* to extract topics from your corpora, and measure each document on how much of each topic it contains. How do you interpret these topics?

Some **potential inspiration** below (but please keep your own story simple!):
* https://liferay.de.dariah.eu/tatom/topic_model_mallet.html covers a few examples of text analysis
* http://fantheory.viacom.com/
* https://pudding.cool/2017/02/vocabulary/

Additional resources on LDA (if you are interested): 
* https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d
* https://www.youtube.com/watch?v=DDq3OVp9dNA

[LDA Code](https://medium.com/mlreview/topic-modeling-with-scikit-learn-e80d33668730)

[Grid Search LDA](https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/)

In [14]:
n_components=2
lda = LatentDirichletAllocation(n_topics=n_components, max_iter=5, learning_method='online',
                                learning_offset=50.,random_state=0,n_jobs=-1).fit(dtm)

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

/usr/local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [15]:
no_top_words = 100
display_topics(lda, vocab, no_top_words)

Topic 0:
movies love films think character life characters watch bad little man seen did know plot does acting end better scene scenes ve say real old years makes work director new 10 actors lot watching world doesn look young funny cast actually didn quite going thing things fact role thought want family horror saw bit performance pretty got comedy times music girl long interesting point right come action feel big day away gets guy especially fun isn woman making ll hard wife original played far beautiful comes sure tv father year sex kind having worth believe job true play dvd shows
Topic 1:
batman charlie holmes pesci series vinny brown bruce car fly frankenstein dracula star wraith sharpe british madeleine ffolkes davis sheen station tomei phantasm woodstock seth cousin kinski howard battle arthur space sisko shaft gang goldblum flynn joe moore joker cronenberg chaney jews brundle north dr episode sea nazi sherlock plays christy thanksgiving washington scientist wayne nick monster 

- Should remove common words like br and film and movie

### Normalize your bags
In the above exercise, you may find it important to normalize your data.  One useful method when dealing with text is *Term Frequency - Inverse Document Frequency (TF-IDF)*. You can see more detail on this here: http://blog.christianperone.com/2011/10/machine-learning-text-feature-extraction-tf-idf-part-ii/.

Once you understand the concept, **express your data as TF-IDF vectors (instead of simple bag-of-words counts), and see if it changes your above story**. 

In [ ]:
dtm_tfid = TfidfTransformer().fit_transform(dtm)
dtm_tfid = dtm_tfid.toarray() 

In [ ]:
getFrequentWord(dtm_tfid)

In [ ]:
doHierarchialClustering(dtm_tfid,'cosine',threshold=0.9)

In [ ]:
doHierarchialClustering(dtm_tfid,'euclidean',threshold=0.8)

### Show us your bags (Version 2)

Show and explain what one of your documents looks like as a TF-IDF vector below.  How is this different from a simple bag-of-words?

In [ ]:
# dtm[vocab=='br']

In [ ]:
np.unique(dtm_tfid[2,:])

In [ ]:
np.unique(dtm[2,:])

In [ ]:
dtm_tfid_sum = dtm_tfid.sum(axis=0)
vocab[dtm_tfid_sum.argmax(axis=0)]

# Chapter 2: Simple Supervised Learning with Text
Now that you are comfortable with treating text as numbers, we can try out supervised learning.  We'll use a labelled dataset of IMDB reviews to classify each review as 'positive' or 'negative'.  You can **find the data below:**

http://ai.stanford.edu/~amaas/data/sentiment/

Load in and process the data, then train a supervised learning model.  **You should achieve val or test set accuracy of 85%**. Pretty good for a simple bag, no?

[Sentiment Analysis in IMDB)[https://itnext.io/machine-learning-sentiment-analysis-of-movie-reviews-using-logisticregression-62e9622b4532]

# Chapter 3: Playing with Recurrent Neural Networks (RNN)
So far, we've only treated text as a simple bag, with reasonable results.  We'll now shift to a more complex representation of language: recurrent neural networks.  To do so, we need to process text at the word or character level, and capture the sequence of a document. 

Our task here is to build an RNN that 'eats up' sequences of characters in order to predict the next character in a sequence, for every step in the sequence of a document. This is a common (and fun) task, with lots of examples available online. 

For this task, use existing RNN APIs (don't code everything from scratch) from Keras or PyTorch. 

**Read up on RNNs and this exercise** below:
* http://karpathy.github.io/2015/05/21/rnn-effectiveness/ - start here!
* https://github.com/martin-gorner/tensorflow-rnn-shakespeare - video, slides and code going through an example with Shakespeare
* http://killianlevacher.github.io/blog/posts/post-2016-03-01/post.html - another nice example based on Trump tweets

### Prepare your data

Our first step is to prepare our text. **Process your corpora into a format that can be used by an RNN, and walkthough one sequence below**.

An **example way to shape your data** for this task is as follows (feel free to play around with different structures):

*In this example your corpora starts with the string 'the cat and I'*
* RNN input: divide your text into sequences of 10 characters e.g. 'the cat an'
* RNN output: the 1 character immediately following RNN input sequences e.g. 'd'. 
* Note: You may or may not want to divide your text into overlapping strings (e.g. RNN input contains 'the cat an', 'he cat and', 'e cat and ', ...) . How is the model different in each case?
* Note: Your 'vocabulary' or `vocab_size` here is the number of unique characters in your text (and therefore the number of classes you want to predict)

### Generate text

Once the model is trained, we can use it to generate completely new text in the style of your training data.  **Train a model using your original choice of corpus below, and generate some sample sentences.** Don't worry too much about your loss / accuracy during training, but instead check on the text your model is generating. Your generated text should be somewhat coherent, i.e. similar to your training text in structure, and not excessively mispelled.

An **example model architecture** is as follows (feel free to play around with different structures):
* Embedding (for each character in your vocab) of dimension 64
* Dropout of 20% for the embedding input to the RNN
* 2 LSTM layers, each of dimension 512 (play around with the number and dimension of hidden layers)
* Dropout of 50% for each LSTM layer
* Dense softmax layer of same dimension as your vocab size (e.g. if your vocab size is 100, this layer is the probabilty that your output is one of 100 possible characters)
    
**You should understand what each of the above elements are and how they work at a high level by the end of this week's exercise.**

### Generalizing the exercise
How do you think you can apply what you learned in the above exercise to other problems involving text? For example, how would you tackle the previous IMDB sentiment classification task using an RNN architecture? **Discuss below.**

(*Bonus*: create an RNN model for the IMDB classification task and discuss your results. How does the performance compare to your bag of words model?)

# Chapter 4: RNNs from scratch
Now that you understand how to use RNNs, it's time to build a basic one from scratch.  You won't understand how they work until you get stuck in the weeds! 

### Generate text (Version 2)
Your task is now to **build the forward pass of a simple RNN, without using any existing RNN APIs**. You can use PyTorch or Tensorflow (Keras is too high level for this exercise), both of which will automatically handle backpropagation for you.  If you use Tensorflow, please research and use Eager execution - it replaces Tensorflow's default graph / session framework, which is very difficult to learn and debug.

Similar to last week's exercise, create a class for your network (write forward and loss steps, allowing PyTorch or Tensorflow to handle backpropagation for you).  Consider appropriate sizes for your input, hidden and output layers - your __init__ method should take in the params `hidden_size`, `vocab_size`, and `embedding_size` (if you use embeddings). Using these variables, you should initialise three weight layers `input_layer`, `hidden_layer`, and `output_layer`.  In an RNN, you will also have to deal with another item - the `hidden_state`. (Note: your RNN structure may vary slightly from this depending on your learning materials, but the key part is always `hidden_state`)

You should **train your RNN on the same data and task as in Chapter 3.**

**How do the results of your basic RNN compare to your model in Chapter 3?**  What do you think explains the difference in performance? Discuss below.

Some relevant resources on LSTMs (and RNN theory) below if you are interested:
* http://colah.github.io/posts/2015-08-Understanding-LSTMs/
* https://www.youtube.com/watch?v=93rzMHtYT_0&list=LLpNVCNE9cYqVrjb2O8bZUGg&index=2&t=0s
* https://www.youtube.com/watch?v=zQxm3Upr3_I
* http://harinisuresh.com/2016/10/09/lstms/

### Bonus Challenges (not required!):
1. Build the forward pass of an LSTM, without using any existing RNN APIs (as above, with PyTorch or Tensorflow)
1. Build a basic RNN or LSTM in Numpy - including forward pass as well as backpropogation